<a href="https://colab.research.google.com/github/eshmaneva/DS-Practical/blob/main/RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Preinstalling necessary libraries / specificated versions are necessary to avoid compatibility issues
!pip install datasets==1.0.2
!pip install transformers

#!rm seq2seq_trainer.py
#!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py

!pip install rouge_score

     |████████████████████████████████| 1.8 MB 4.1 MB/s 
     |████████████████████████████████| 212 kB 92.8 MB/s 
     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 
     |████████████████████████████████| 6.6 MB 63.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import datasets
import transformers
import pandas as pd
from datasets import Dataset

#Tokenizer
from transformers import RobertaTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Installing seq2seq training library & class
!git clone https://github.com/google/seq2seq.git
!pip install -e seq2seq
import seq2seq

#Training
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer
from dataclasses import dataclass, field
from typing import Optional

Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 5.12 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Obtaining file:///content/seq2seq
     |████████████████████████████████| 60 kB 3.0 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191621 sha256=3e5bcb12a99f36e34be26c0aa3331fedd7f5312beb93c7aaccd5c1e25b1e6b66
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/ffb9a1f51b2b00fee42e7f67f5a5d8e10c67d048cda09ccd57
Successfully built pyrouge
  Running setup.py develop for seq2seq


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/DS Practical/Reviews.csv", engine="python", error_bad_lines=False)
df.drop(columns=['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator', 'Score', 'Time'],axis=1,inplace=True)
print("Before",len(df))
df = df.dropna()
print("Data size:",len(df))
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Before 568454
Data size: 568427


,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
#Shortening the data for testing purposes (remove the whole cell for full training)
df1=df.loc[1:200]
df1.head()
print("Data size:",len(df1))

Data size: 200


In [6]:
#Converting Pandas DataFrame into the 'dataset' library 
from datasets import Dataset
import numpy as np
n_train = int(np.round(df1.shape[0]*0.8))
n_val = int(np.round(df1.shape[0]*0.1))
n_test = int(np.round(df1.shape[0]*0.1))
train_data=Dataset.from_pandas(df1[:n_train])
val_data=Dataset.from_pandas(df1[n_train:n_train+n_val])
test_data=Dataset.from_pandas(df1[n_train+n_val:n_train+n_val+n_test])

#train_data=Dataset.from_pandas(df[:550000])
#val_data=Dataset.from_pandas(df[550000:555000])
#test_data=Dataset.from_pandas(df[556000:557000])

In [7]:
#Tokenizing the data
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
batch_size=16  #change to 16 for full training
encoder_max_length=40
decoder_max_length=6

def process_data_to_model_inputs(batch):
  inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  #RoBERTa automatically shifts the labels, so, the labels correspond exactly to `decoder_input_ids` 
  #We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

#Processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

#Processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
from transformers import EncoderDecoderModel

# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("roberta-base", "roberta-base", tie_encoder_decoder=True)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.7.crossattention.self.key.weight', 'roberta.encoder.layer.5.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.query.bias

In [9]:
#Set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

#Sensible parameters for beam search
#Set decoding params     
roberta_shared.config.pad_token_id = tokenizer.pad_token_id                          
roberta_shared.config.max_length = 40
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size
from transformers import TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [10]:
#Load rouge for validation
rouge=datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids=pred.label_ids
    pred_ids=pred.predictions

    # all unnecessary tokens are removed
    pred_str=tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids==-100]=tokenizer.pad_token_id
    label_str=tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output=rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision,4),
        "rouge2_recall": round(rouge_output.recall,4),
        "rouge2_fmeasure": round(rouge_output.fmeasure,4),
    }

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

In [11]:
'''
#Load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    #All unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }
'''

'\n#Load rouge for validation\nrouge = datasets.load_metric("rouge")\n\ndef compute_metrics(pred):\n    labels_ids = pred.label_ids\n    pred_ids = pred.predictions\n\n    #All unnecessary tokens are removed\n    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)\n    labels_ids[labels_ids == -100] = tokenizer.pad_token_id\n    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)\n\n    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid\n\n    return {\n        "rouge2_precision": round(rouge_output.precision, 4),\n        "rouge2_recall": round(rouge_output.recall, 4),\n        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),\n    }\n'

In [12]:
#Parameter settings v.2
training_args = Seq2SeqTrainingArguments(   
    output_dir="/content/drive/MyDrive/DS Practical/",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    evaluation_strategy="epoch", #"steps"
    do_train=True,
    do_eval=True,
    logging_steps=2, 
    save_steps=16,
    eval_steps=4,
    warmup_steps=1,  
    #max_steps=400, # 5000
    num_train_epochs=50,
    overwrite_output_dir=True,
    save_total_limit=2,
    fp16=False,
    no_cuda=False,
    seed=42,
    #resume_from_checkpoint=os.path.join(path, 'pretrained_roberta&'),
    dataloader_num_workers=3,
    logging_strategy='epoch'
)

#Parameter settings v.1
'''
training_args = Seq2SeqTrainingArguments(
    Parameter settings v.1
    output_dir="/content/drive/MyDrive/DS Practical/",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=2,  
    save_steps=16, 
    eval_steps=500, 
    warmup_steps=500,  
    #max_steps=150, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=1,
    fp16=False,
    no_cuda=False,
    seed=42,
    dataloader_num_workers=3,
    resume_from_checkpoint=True,
    #os.path.join("/content/drive/MyDrive/DS Practical/",'checkpoint-7056')
    '''

#Instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data
)

In [13]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 160
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 500
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarning: Version v4.12.0 introduces a better way to train encoder

Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
1,8.600600,6.384942,0.000000,0.000000,0.000000
2,6.145100,6.234996,0.000000,0.000000,0.000000
3,5.698300,6.130759,0.000000,0.000000,0.000000
4,5.457800,6.059879,0.000000,0.000000,0.000000
5,5.199600,6.212119,0.000000,0.000000,0.000000
6,4.978200,6.512440,0.000000,0.000000,0.000000
7,4.770300,6.453692,0.000000,0.000000,0.000000
8,4.482400,6.639272,0.000000,0.000000,0.000000
9,4.081600,7.162561,0.000000,0.000000,0.000000
10,3.833200,6.878657,0.000000,0.000000,0.000000


The following columns in the evaluation set  don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 20
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/DS Practical/checkpoint-16
Configuration saved in /content/drive/MyDrive/DS Practical/checkpoint-16/config.json
Model weights saved in /content/drive/MyDrive/DS Practical/checkpoint-16/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/DS Practical/checkpoint-240] due to args.save_total_limit
Deleting older checkpoint [/content/drive/MyDrive/DS Practical/checkpoint-256] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:530: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by

TrainOutput(global_step=500, training_loss=2.6936133441925048, metrics={'train_runtime': 359.8659, 'train_samples_per_second': 22.231, 'train_steps_per_second': 1.389, 'total_flos': 220139199360000.0, 'train_loss': 2.6936133441925048, 'epoch': 50.0})

In [14]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
roberta_shared.save_pretrained("/content/drive/MyDrive/DS Practical/checkpoints")

model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/DS Practical/checkpoints")
#model.to("cuda")
batch_size = 32

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=50, return_tensors="pt")
    input_ids = inputs.input_ids#.to("cuda")
    attention_mask = inputs.attention_mask #.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
test_data = test_data.select(range(5000))

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])

pred_str = results["pred"]
label_str = results["Summary"]

loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggin

  0%|          | 0/157 [00:00<?, ?ba/s]

IndexError: ignored

In [ ]:
#1000
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE F SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)